In [174]:
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import plotly.express as px

In [175]:
import numpy as np
import pandas as pd

In [176]:
import pulp

In [177]:
f_conv_mon = 0.0008

In [178]:
C = np.array([[105, 1267, 1134, 2758, 515, 1350, 1311, 1750, 1565, 1137, 3110, 1365]]) #dinar

C = C * f_conv_mon

#C = np.concatenate((C[:, 0:1], C[:, 2:4], C[:, 5:11]), axis=1)

C = np.concatenate((C, C), axis=1)


In [179]:
C = np.concatenate(( np.concatenate((C, np.zeros(C.shape)), axis=1), 
                     np.concatenate((np.zeros((1, 24)), np.ones((1, 24))), axis=1),
                     np.concatenate((np.ones((1, 24)), np.zeros((1, 24))), axis=1)
                    ), axis=0)

In [180]:
A = np.array([  [     1,       1,       1,       1,       1,       1,       0,       0,       0,       0,       0,       0],
                [     0,       0,       0,       0,       0,       0,       1,       1,       1,       1,       1,       1],
                [   100,      50,      50,      50,     100,     100,      50,     100,     100,     100,     100,     100],
                [   100,      50,      50,      25,      50,     150,      50,     100,     100,      50,      50,      50],
                [     0,       0,       0,       0,       0,      12,       0,       0,      12,       0,       0,       0],
                [     1,       1,       2,       1,       1,       3,       1,       2,       2,       2,       3,       1],
                [     6,      29,      32,      14,      54,      60,       0,       0,      18,      13,      29,       8],
                [     2,       0,      17,      23,       0,       0,      22,       6,      50,     102,      77,      31],
                [     0,       0,       0,       0,       0,      12,      35,      24,      80,       0,     112,      58],
                [     9,       8,       7,      34,      26,      22,       0,       0,      52,       0,      43,       0],
                [   1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5,     1.5],
                [     1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1,       1],
                [   0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5,     0.5],
                [   0.5,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0,       0],
                [   390,     355,     370,     430,     440,       0,     275,       0,     100,     133,     213,     100],
                [   275,       0,     470,     305,       0,       0,    1430,     188,    1015,     955,    1343,    1215],
                [     0,       0,     150,       0,       0,     750,     443,     840,    1683,       0,    1748,    1095],
                [   300,     295,     305,     303,     210,     860,       0,       0,       0,       0,       0,       0],
                [345000,  233000,  216000,  242000,  360000,  650000,  244000,  242000,  310000,  263000,  250000,  285000]])

In [181]:
b = np.array([2395,2260,419190,397560,7000,13075,50593,554800,562400,562400,103056,103056,103056,954,20699716,20929709,21159705,21159705,1445855000]) #dinar

In [182]:
b[18] = b[18] * f_conv_mon
A[18,:] = A[18,:] * f_conv_mon

In [183]:
A = np.concatenate((np.concatenate((A, np.zeros(A.shape)), axis=1), 
                    np.concatenate((np.zeros(A.shape), A), axis=1)))

In [184]:
A.shape

(38, 24)

In [185]:
nX = 12
_A = np.zeros((2, A.shape[1]))
_A[0, 0] = _A[1, 2] = _A[1, 3] =  -1
_A[0, nX+2] = _A[0, nX+3] = _A[1, nX+7] = 1

In [186]:
A = np.concatenate((A, _A), axis=0)

In [187]:
b = np.concatenate((b, b, np.array([0, 0])), axis=0)

In [188]:
A = np.concatenate((A, np.zeros(A.shape)), axis=1)

In [189]:
A.shape

(40, 48)

In [190]:
I = np.identity(24)
w1 = 113 # min 5%
w2 = -(10**10)

A = np.concatenate((A, np.concatenate((-I, w1*I), axis=1), np.concatenate((I, w2*I), axis=1)), axis=0)

In [191]:
A.shape

(88, 48)

In [192]:
b = np.concatenate((b, np.zeros((A.shape[0] - b.shape[0]))), axis=0)

In [193]:
I = np.identity(12)
_A = np.concatenate((I, I), axis=1)
_A = np.concatenate((_A, np.zeros((_A.shape[0], A.shape[1] - _A.shape[1]))), axis=1)
A = np.concatenate((A, _A), axis=0)

In [194]:
b = np.concatenate((b, np.ones((6,))*2395, np.ones((6,))*2260), axis=0)

In [195]:
sig = lambda sigma, NUM_ITER: 1/(1 + np.exp(sigma*(np.linspace(0, NUM_ITER, NUM_ITER)-(NUM_ITER/2))))

alpha_lin = np.linspace(0, 1, 100)
alpha_sig_004 = sig(-0.004, 1000)
alpha_sig_06 = sig(-0.06, 1000)

In [196]:
l_alpha = {'alpha_lin':     alpha_lin, 
           #'alpha_sig_004': alpha_sig_004, 
           #'alpha_sig_06':  alpha_sig_06,
           }

In [197]:
def create_var():
    var = np.array([])

    lbx = [0] * 24
    #lbx[0] = 1500
    #lbx[12] = 1500

    for j in range(1, 3):
        for i in range(1, 13):
            var = np.append(var, np.array([pulp.LpVariable("x_{}_{:02d}".format(j, i), lowBound = lbx[i-1], upBound=None, cat='Integer')]))

    for j in range(1, 3):
        for i in range(1, 13):
            var = np.append(var, np.array([pulp.LpVariable("y_{}_{:02d}".format(j, i), cat='Binary')]))
    
    return var

In [198]:
z1p = 14101.4512
z1n = 0

_z1 = z1p-z1n

z2p = 24
z2n = 0

_z2 = z2p-z2n

z3p = 9310.0
z3n = 0

_z3 = z3p-z3n

In [199]:
def objective(alpha):
    var = create_var()

    Fs = np.array([])

    for c in C:
        Fs = np.append(Fs, c.dot(var))

    fo = alpha*((Fs[0] - z1n)/_z1) + (1 - alpha) * ((Fs[1] -z2n)/(_z2*2) + (Fs[2] - z3n)/(_z3*2))

    return (var, Fs, fo)

In [200]:
def print_solution(sol, linearProblem, X):
    saida = "\nStatus = {} \nValue = {}".format(pulp.LpStatus[sol], pulp.value(linearProblem.objective))
    
    for i in range(0, X.shape[0]):
        saida += "\n{} = {}".format(X[i], pulp.value(X[i]))
    
    print(saida)

In [201]:
columns=["alpha_type", "iter", "alpha", "status"]
for j in range(1, 3):
        for i in range(1, 13):
            columns.append("X_{}_{}".format(j, i))
for j in range(1, 3):
        for i in range(1, 13):
            columns.append("Y_{}_{}".format(j, i))
columns = columns + ["obj_value", "f1", "f2", "f3"]

In [202]:
solution_lst = []
for key in l_alpha:
    aph = l_alpha[key]
    for i_alpha in range(0, len(aph)):            
        alpha = aph[i_alpha]
        d_var, F, f = objective(alpha)
        model = pulp.LpProblem("First_Model", pulp.LpMaximize)
        #model = pulp.LpProblem("First_Model", pulp.LpMinimize)

        model += f 

        Ax = A.dot(d_var)

        for i in range(0, Ax.shape[0]):
            model += Ax[i] <= b[i]
        
        solution = model.solve(pulp.GLPK(path='F:\glpk-4.65\w64\glpsol.exe'))
        #solution = model.solve()

        X = np.array([pulp.value(i) for i in d_var])

        fs = C.dot(X)

        solution_lst.append([key,
                            i_alpha,
                            alpha,
                            str(pulp.LpStatus[solution])] +
                            X.tolist() +
                            [pulp.value(model.objective),
                            fs[0],
                            fs[1],
                            fs[2],
                            ])
    #print_solution(sol=solution, linearProblem=model, X=d_var)

In [ ]:
dt_all = pd.DataFrame(solution_lst, columns=columns)

In [ ]:
#from unicodedata import decimal
#df = pd.DataFrame(b.transpose())
#df.to_clipboard(index=False,header=False, decimal=',')

In [ ]:
dt = dt_all[dt_all.alpha_type == 'alpha_lin']

In [ ]:
#sns.scatterplot(data=dt, x = 'alpha', y='obj_value')

fig = px.scatter(dt, 
              x ='alpha', 
              y='obj_value', 
              title='Objective Function (z<sub>1y</sub>) versus λ',
              #color='alpha_type',
              labels={'alpha': 'λ',
                      'obj_value': 'z'})
                      
fig.update_traces(marker=dict(color='rgb(115, 115, 115)',
                              size=3
                              )
                    )

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
#fig['data'][0]['line']['color']='rgb(115, 115, 115)'
#fig['data'][0]['line']['width']=3

fig.show()
#fig.write_image('images/fig_mdl2_z_vs_lambda.png')

In [ ]:
#sns.lineplot(data=dt, x = 'alpha', y='f1')

#sns.lineplot(data=dt, x = 'alpha', y='f2')

#sns.lineplot(data=dt, x = 'alpha', y='f3')

fig = px.line(dt, 
              x ='alpha', 
              y='f1', 
              title='Net Income of Farm (z<sub>1</sub>) versus λ',
              #color='alpha_type',
              labels={'alpha': 'λ',
                      #'alpha_type': 'λ',
                      'f1': 'z<sub>1</sub> (USD)'})
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        dtick=500,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
fig['data'][0]['line']['color']='rgb(115, 115, 115)'
fig['data'][0]['line']['width']=3

fig.show()
#fig.write_image('images/fig_mdl2_z2y_vs_lambda.png')

In [ ]:
#sns.lineplot(data=dt, x = 'alpha', y='f2')

#sns.lineplot(data=dt, x = 'alpha', y='f3')

fig = px.line(dt, 
              x ='alpha', 
              y='f2', 
              title='Number of Different Crops Grown (z<sub>2</sub>) versus λ',
              #color='alpha_type',
              labels={'alpha': 'λ',
                      'f2': 'z<sub>2</sub>'})
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
fig['data'][0]['line']['color']='rgb(115, 115, 115)'
fig['data'][0]['line']['width']=3

fig.show()
#fig.write_image('images/fig_mdl2_z2_vs_lambda.png')

In [ ]:
#sns.lineplot(data=dt, x = 'alpha', y='f3')

fig = px.line(dt, 
              x ='alpha', 
              y='f3', 
              title='Used Area (z<sub>3</sub>) versus λ',
              #color='alpha_type',
              labels={'alpha': 'λ',
                      'f3': 'z<sub>3</sub>'})
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
fig['data'][0]['line']['color']='rgb(115, 115, 115)'
fig['data'][0]['line']['width']=3

fig.show()
#fig.write_image('images/fig_mdl2_z3_vs_lambda.png')